In [5]:
!pip install transformers pandas matplotlib scikit-learn --force

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.5/104.5 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 865.2/865.2 MB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 133.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 99.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://download.pytorch.org/whl/cu118
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 955.6/955.6 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 156.5/156.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 117.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 100.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 106.4 MB/s eta 0:00:00
  Attempting uninstall: triton
    Found existing installation: triton 3.2.0
    Uninstalling triton-3.2.0:
      Successfully uninstalled triton-3.2.0
  Attempting uninstall: sympy
    Found existing installation: sympy 1.13.1
    Uninstalling sympy-1.13.1:
      Successfully uninstalled sympy-1.13.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
fastai 2.7.19 requires torch<2.7,>=1.10, but you have torch 2.7.0+cu1

In [1]:
import os
import logging
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, get_linear_schedule_with_warmup
from torch.optim import AdamW
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tqdm import tqdm

# Speed up
torch.backends.cudnn.benchmark = True

# Disable tokenizers parallelism warning
os.environ['TOKENIZERS_PARALLELISM'] = 'false'

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")
if torch.cuda.is_available():
    logger.info(f"GPU: {torch.cuda.get_device_name(0)}")

# Dataset Class
class UnicodePreetiFontDataset(Dataset):
    def __init__(self, unicode_texts, preeti_texts, tokenizer, max_length=128):
        self.unicode_texts = unicode_texts
        self.preeti_texts = preeti_texts
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.unicode_texts)

    def __getitem__(self, idx):
        unicode_text = str(self.unicode_texts[idx])
        preeti_text = str(self.preeti_texts[idx])

        encoding = self.tokenizer(
            unicode_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        target_encoding = self.tokenizer(
            preeti_text,
            truncation=True,
            padding='max_length',
            max_length=self.max_length,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].squeeze(),
            'attention_mask': encoding['attention_mask'].squeeze(),
            'labels': target_encoding['input_ids'].squeeze()
        }

# Accuracy calculation
def calculate_accuracy(preds, labels):
    preds = preds.view(-1)
    labels = labels.view(-1)
    mask = labels != -100
    correct = (preds[mask] == labels[mask]).sum().item()
    total = mask.sum().item()
    return correct / total if total > 0 else 0

# Plotting function
def plot_metrics(epochs, train_losses, val_losses, train_accuracies, val_accuracies):
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Train Loss', color='blue')
    plt.plot(epochs, val_losses, label='Validation Loss', color='red')
    plt.title('Loss over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracies, label='Train Accuracy', color='green')
    plt.plot(epochs, val_accuracies, label='Validation Accuracy', color='orange')
    plt.title('Accuracy over Epochs')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.tight_layout()
    plt.savefig('training_metrics_plot.png')
    plt.close()

# Load CSV dataset
def load_dataset(path, sample_fraction=1.0):
    df = pd.read_csv(path, encoding='utf-8')
    df = df.sample(frac=sample_fraction, random_state=42)

    if 'unicode' not in df.columns or 'preeti' not in df.columns:
        raise ValueError("CSV must contain 'unicode' and 'preeti' columns")

    return df['unicode'].tolist(), df['preeti'].tolist()

# Model Training
def train_model(unicode_texts, preeti_texts, model_name='facebook/nllb-200-distilled-600M', epochs=5):
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name).to(device)

    train_texts, val_texts, train_labels, val_labels = train_test_split(unicode_texts, preeti_texts, test_size=0.2, random_state=42)

    train_dataset = UnicodePreetiFontDataset(train_texts, train_labels, tokenizer)
    val_dataset = UnicodePreetiFontDataset(val_texts, val_labels, tokenizer)

    train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=8)

    optimizer = AdamW(model.parameters(), lr=3e-5)
    total_steps = len(train_loader) * epochs
    scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=int(0.1 * total_steps), num_training_steps=total_steps)

    train_losses, val_losses = [], []
    train_accuracies, val_accuracies = [], []
    epoch_list = []

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        total_acc = 0

        for batch in tqdm(train_loader, desc=f"Epoch {epoch+1}/{epochs} - Training"):
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad()

            total_loss += loss.item()

            preds = outputs.logits.argmax(dim=-1)
            acc = calculate_accuracy(preds, labels)
            total_acc += acc

        avg_train_loss = total_loss / len(train_loader)
        avg_train_acc = total_acc / len(train_loader)

        # Validation
        model.eval()
        val_loss = 0
        val_acc = 0

        with torch.no_grad():
            for batch in tqdm(val_loader, desc=f"Epoch {epoch+1}/{epochs} - Validation"):
                input_ids = batch['input_ids'].to(device)
                attention_mask = batch['attention_mask'].to(device)
                labels = batch['labels'].to(device)

                outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
                val_loss += outputs.loss.item()

                preds = outputs.logits.argmax(dim=-1)
                val_acc += calculate_accuracy(preds, labels)

        avg_val_loss = val_loss / len(val_loader)
        avg_val_acc = val_acc / len(val_loader)

        logger.info(f"Epoch {epoch+1} | Train Loss: {avg_train_loss:.4f} | Val Loss: {avg_val_loss:.4f} | Train Acc: {avg_train_acc:.4f} | Val Acc: {avg_val_acc:.4f}")

        epoch_list.append(epoch+1)
        train_losses.append(avg_train_loss)
        val_losses.append(avg_val_loss)
        train_accuracies.append(avg_train_acc)
        val_accuracies.append(avg_val_acc)

    plot_metrics(epoch_list, train_losses, val_losses, train_accuracies, val_accuracies)

    model.save_pretrained("unicode_to_preeti_model")
    tokenizer.save_pretrained("unicode_to_preeti_model")

    return model, tokenizer

# Inference
def inference(text, model, tokenizer):
    inputs = tokenizer(text, return_tensors='pt', truncation=True, max_length=128).to(device)
    outputs = model.generate(inputs.input_ids, max_length=128, num_beams=4)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Main function
def main():
    unicode_texts, preeti_texts = load_dataset("/content/drive/MyDrive/thesis/phrase_dataset_unicode_to_preeti.csv", sample_fraction=1.0)

    model, tokenizer = train_model(unicode_texts, preeti_texts, epochs=5, model_name = "Helsinki-NLP/opus-mt-en-ROMANCE")

    print("\n--- Inference Samples ---")
    for i in range(3):
        sample = unicode_texts[i]
        result = inference(sample, model, tokenizer)
        print(f"Unicode: {sample}")
        print(f"Preeti : {result}\n")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/312M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

Epoch 1/5 - Training:   0%|          | 0/2500 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/312M [00:00<?, ?B/s]

Epoch 5/5 - Validation: 100%|██████████| 625/625 [00:39<00:00, 15.97it/s]
/usr/local/lib/python3.11/dist-packages/transformers/modeling_utils.py:3339: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 512, 'num_beams': 4, 'bad_words_ids': [[65000]]}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(



--- Inference Samples ---
Unicode: गर्ने
Preeti : ug]{

Unicode: चीन जस्ता देशहरुले
Preeti : rLg h:tf b]zx?n]

Unicode: आदिलाई
Preeti : cflbnfO{



In [7]:
import os
import torch
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import logging
import pandas as pd

# Logging setup
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s: %(message)s')
logger = logging.getLogger(__name__)

# Device setup
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
logger.info(f"Using device: {device}")

class UnicodePreetiFontConverter:
    def __init__(self, model_path="/content/drive/MyDrive/thesis/unicode_to_preeti_model"):
        """
        Initialize the converter with pre-trained model

        Args:
            model_path (str): Path to saved model directory
        """
        try:
            # Load tokenizer and model
            self.tokenizer = AutoTokenizer.from_pretrained(model_path)
            self.model = AutoModelForSeq2SeqLM.from_pretrained(model_path).to(device)
            self.model.eval()  # Set to evaluation mode

            logger.info("Model loaded successfully!")
        except Exception as e:
            logger.error(f"Model loading error: {e}")
            raise

    def convert_single(self, unicode_text, max_length=128):
        """
        Convert single Unicode text to Preeti

        Args:
            unicode_text (str): Input Unicode text
            max_length (int): Maximum length for tokenization

        Returns:
            str: Converted Preeti text
        """
        try:
            # Prepare input
            inputs = self.tokenizer(
                unicode_text,
                return_tensors='pt',
                truncation=True,
                max_length=max_length
            ).to(device)

            # Generate translation
            outputs = self.model.generate(
                inputs.input_ids,
                max_length=max_length,
                num_beams=4,  # Beam search for better results
                early_stopping=True
            )

            # Decode output
            preeti_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
            return preeti_text

        except Exception as e:
            logger.error(f"Conversion error for text '{unicode_text}': {e}")
            return None

    def batch_convert(self, unicode_texts, max_length=128):
        """
        Convert multiple Unicode texts to Preeti

        Args:
            unicode_texts (list): List of Unicode texts
            max_length (int): Maximum length for tokenization

        Returns:
            list: List of converted Preeti texts
        """
        return [self.convert_single(text, max_length) for text in unicode_texts]

    def convert_from_csv(self, csv_path, unicode_column='unicode', batch_size=32):
        """
        Convert Unicode texts from CSV file

        Args:
            csv_path (str): Path to CSV file
            unicode_column (str): Name of Unicode column
            batch_size (int): Number of texts to process in each batch

        Returns:
            pd.DataFrame: DataFrame with original and converted texts
        """
        try:
            # Read CSV
            df = pd.read_csv(csv_path, encoding='utf-8')

            if unicode_column not in df.columns:
                raise ValueError(f"Column '{unicode_column}' not found in CSV")

            # Convert texts in batches
            converted_texts = []
            for i in range(0, len(df), batch_size):
                batch = df[unicode_column][i:i+batch_size].tolist()
                converted_batch = self.batch_convert(batch)
                converted_texts.extend(converted_batch)

            # Add converted column
            df['preeti_converted'] = converted_texts

            return df

        except Exception as e:
            logger.error(f"CSV conversion error: {e}")
            return None

def main():
    # Initialize converter
    converter = UnicodePreetiFontConverter()

    # Single text conversion examples
    print("\n--- Single Text Conversion ---")
    test_texts = ["क", "का", "कि", "की"]
    for text in test_texts:
        preeti_text = converter.convert_single(text)
        print(f"Unicode: {text} | Preeti: {preeti_text}")

    # CSV conversion example
    print("\n--- CSV Batch Conversion ---")
    csv_path = 'sample_unicode_texts.csv'

    # Ensure sample CSV exists
    if not os.path.exists(csv_path):
        sample_data = {
            'unicode': ['क', 'का', 'कि', 'की', 'कु', 'के', 'अ', 'इ', 'उ'],
            'description': ['Sample Nepali Character', 'ka', 'ki', 'kI', 'ku', 'ke', 'a', 'i', 'u']
        }
        pd.DataFrame(sample_data).to_csv(csv_path, index=False)
        print(f"Created sample CSV: {csv_path}")

    # Convert CSV
    converted_df = converter.convert_from_csv(csv_path)
    if converted_df is not None:
        print("\nConverted DataFrame:")
        print(converted_df)

        # Optional: Save converted results
        converted_df.to_csv('converted_unicode_to_preeti.csv', index=False)
        print("Saved converted results to 'converted_unicode_to_preeti.csv'")

from IPython.display import display, HTML

def display_preeti_text(text):
    html = f"""
    <style>
    @font-face {{
        font-family: 'Preeti';
        src: url('Preeti.ttf') format('truetype');
    }}
    .preeti-text {{
        font-family: 'Preeti';
        font-size: 20px;
    }}
    </style>
    preeti font:<div class="preeti-text"> {text}</div></br>
    """
    display(HTML(html))




In [8]:
def advanced_text_processing_demo():
    """
    Advanced text processing demonstration
    """
    converter = UnicodePreetiFontConverter()

    # Complex text processing scenarios
    complex_texts = [
        "नमस्ते",  # Greeting
        "डाटा विज्ञान",  # Data Science
        "कम्प्युटर",  # Computer
        "मेराे नाम सुमन हाे । what is yours?",
        "मेराे नाम पुजन हाे ।  तिमी के गछाै ",
        "गाराे छ केटा हाे । स्यालरी कहिले अाउने हाे "
    ]

    print("\n--- Advanced Text Processing ---")
    for text in complex_texts:
        preeti_text = converter.convert_single(text)
        print(f"Unicode: {text}")
        print(f"Preeti : {preeti_text}\n")
        predicted_text = display_preeti_text(preeti_text)

if __name__ == "__main__":
    main()
    advanced_text_processing_demo()

/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



--- Single Text Conversion ---
Unicode: क | Preeti: s
Unicode: का | Preeti: sf
Unicode: कि | Preeti: ls
Unicode: की | Preeti: sL

--- CSV Batch Conversion ---
Created sample CSV: sample_unicode_texts.csv

Converted DataFrame:
  unicode              description preeti_converted
0       क  Sample Nepali Character                s
1      का                       ka               sf
2      कि                       ki               ls
3      की                       kI               sL
4      कु                       ku               s'
5      के                       ke               s]
6       अ                        a                c
7       इ                        i                O
8       उ                        u                p
Saved converted results to 'converted_unicode_to_preeti.csv'


/usr/local/lib/python3.11/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")



--- Advanced Text Processing ---
Unicode: नमस्ते
Preeti : gd:t]



Unicode: डाटा विज्ञान
Preeti : 8f6f lj1fg



Unicode: कम्प्युटर
Preeti : sDKo'6/



Unicode: मेराे नाम सुमन हाे । what is yours?
Preeti : d]/f] gfd ;'dg xf] wf]



Unicode: मेराे नाम पुजन हाे ।  तिमी के गछाै 
Preeti : d]/f] gfd k'hg xf] wf] ltdL s] u5f}



Unicode: गाराे छ केटा हाे । स्यालरी कहिले अाउने हाे 
Preeti : uf/f] 5 s]6f xf] wf] ;ofn/L slxn] cfpg] xf]



In [9]:
def test_converter_metrics():
    """
    Test the converter using CER (Character Error Rate) and BLEU score metrics
    """
    import nltk
    from nltk.translate.bleu_score import sentence_bleu
    nltk.download('punkt')

    # Initialize converter
    converter = UnicodePreetiFontConverter()

    # Load test dataset with known ground truth
    test_data = [
        {"unicode": "क", "expected_preeti": "s"},
        {"unicode": "का", "expected_preeti": "sf"},
        {"unicode": "कि", "expected_preeti": "ls"},
        {"unicode": "नमस्ते", "expected_preeti": "gd:t]"}
    ]

    # Calculate metrics
    cer_total = 0
    bleu_scores = []

    print("\n--- CER/BLEU Evaluation ---")
    print(f"{'Unicode':<15} {'Expected':<15} {'Predicted':<15} {'CER':<10} {'BLEU':<10}")
    print("-" * 65)

    for item in test_data:
        unicode_text = item["unicode"]
        expected = item["expected_preeti"]
        predicted = converter.convert_single(unicode_text)

        # Calculate Character Error Rate (CER)
        edit_distance = nltk.edit_distance(predicted, expected)
        cer = edit_distance / max(len(expected), 1)
        cer_total += cer

        # Calculate BLEU score
        reference = [list(expected)]
        candidate = list(predicted)
        bleu = sentence_bleu(reference, candidate)
        bleu_scores.append(bleu)

        print(f"{unicode_text:<15} {expected:<15} {predicted:<15} {cer:.4f}      {bleu:.4f}")

    # Calculate average metrics
    avg_cer = cer_total / len(test_data)
    avg_bleu = sum(bleu_scores) / len(bleu_scores)

    print("-" * 65)
    print(f"Average CER: {avg_cer:.4f} (lower is better)")
    print(f"Average BLEU: {avg_bleu:.4f} (higher is better)")

    return avg_cer, avg_bleu

In [10]:
def test_qualitative_assessment():
    """
    Perform qualitative assessment of the converter
    """
    import matplotlib.pyplot as plt
    from matplotlib.font_manager import FontProperties

    # Initialize converter
    converter = UnicodePreetiFontConverter()

    # Sample texts of varying complexity
    test_samples = [
        {"category": "Simple", "texts": ["क", "ख", "ग", "घ"]},
        {"category": "Medium", "texts": ["नमस्ते", "कस्तो छ", "धन्यवाद"]},
        {"category": "Complex", "texts": ["नेपाली भाषा", "कम्प्युटर विज्ञान", "प्रविधि र विकास"]}
    ]

    print("\n--- Qualitative Assessment ---")

    # Process each category
    for category in test_samples:
        print(f"\nCategory: {category['category']}")
        print(f"{'Unicode':<20} {'Preeti':<20}")
        print("-" * 40)

        for text in category["texts"]:
            converted = converter.convert_single(text)
            print(f"{text:<20} {converted:<20}")

    # Visual comparison of sample texts
    plt.figure(figsize=(12, 8))
    plt.subplot(1, 1, 1)
    plt.title("Qualitative Assessment Results")
    plt.axis('off')

    y_position = 0.95
    plt.text(0.1, y_position, "Unicode", fontsize=14, fontweight='bold')
    plt.text(0.5, y_position, "Preeti (Converted)", fontsize=14, fontweight='bold')
    y_position -= 0.05

    # Add samples to plot
    for i, category in enumerate(test_samples):
        y_position -= 0.05
        plt.text(0.02, y_position, f"{category['category']}:", fontsize=12, fontweight='bold')
        y_position -= 0.03

        for text in category["texts"]:
            converted = converter.convert_single(text)
            plt.text(0.1, y_position, text, fontsize=11)
            plt.text(0.5, y_position, converted, fontsize=11)
            y_position -= 0.03

    plt.tight_layout()
    plt.savefig('qualitative_assessment.png')
    plt.close()
    print("\nQualitative assessment visualization saved as 'qualitative_assessment.png'")

In [11]:
def test_performance_benchmark():
    """
    Benchmark the performance of the converter
    """
    import time
    import numpy as np
    import matplotlib.pyplot as plt

    # Initialize converter
    converter = UnicodePreetiFontConverter()

    # Test different input lengths
    test_cases = [
        {"type": "Single Character", "text": "क"},
        {"type": "Word", "text": "नेपाली"},
        {"type": "Short Sentence", "text": "नमस्ते। कस्तो छ?"},
        {"type": "Medium Sentence", "text": "आज नेपालमा धेरै मानिसहरू छन्।"},
        {"type": "Long Sentence", "text": "नेपाल एक सुन्दर देश हो जहाँ विभिन्न संस्कृति, भाषा र परम्परा पाइन्छ। यहाँको प्राकृतिक सौन्दर्य संसारभरि प्रसिद्ध छ।"}
    ]

    # Record processing times
    processing_times = []
    text_lengths = []
    text_types = []

    print("\n--- Performance Benchmark ---")
    print(f"{'Type':<20} {'Length':<10} {'Time (ms)':<15}")
    print("-" * 45)

    for case in test_cases:
        text_type = case["type"]
        text = case["text"]
        text_length = len(text)

        # Measure performance (average of 5 runs)
        times = []
        for _ in range(5):
            start_time = time.time()
            _ = converter.convert_single(text)
            end_time = time.time()
            times.append((end_time - start_time) * 1000)  # convert to ms

        avg_time = np.mean(times)

        print(f"{text_type:<20} {text_length:<10} {avg_time:.2f}")

        text_types.append(text_type)
        text_lengths.append(text_length)
        processing_times.append(avg_time)

    # Create performance visualization
    plt.figure(figsize=(10, 6))
    bars = plt.bar(text_types, processing_times, color='skyblue')

    # Add text lengths on top of bars
    for i, bar in enumerate(bars):
        plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                f'{text_lengths[i]} chars', ha='center', fontsize=9)

    plt.title('Conversion Performance by Text Type')
    plt.xlabel('Text Type')
    plt.ylabel('Processing Time (ms)')
    plt.xticks(rotation=45, ha='right')
    plt.tight_layout()
    plt.savefig('performance_benchmark.png')
    plt.close()

    print("\nPerformance benchmark visualization saved as 'performance_benchmark.png'")

In [12]:
def run_comprehensive_tests():
    """
    Run all tests in a comprehensive test suite
    """
    print("\n========== UNICODE TO PREETI CONVERTER TESTING ==========")

    # Test 1: Metrics Evaluation
    print("\n[TEST 1] CER/BLEU Metrics Evaluation")
    cer, bleu = test_converter_metrics()

    # Test 2: Qualitative Assessment
    print("\n[TEST 2] Qualitative Assessment")
    test_qualitative_assessment()

    # Test 3: Performance Benchmark
    print("\n[TEST 3] Performance Benchmark")
    test_performance_benchmark()

    # Generate visual analysis with image
    generate_test_summary(cer, bleu)

    print("\n========== TESTING COMPLETED ==========")

def generate_test_summary(cer, bleu):
    """
    Generate and visualize a summary of all test results
    """
    import matplotlib.pyplot as plt

    # Create a visual summary of test results
    plt.figure(figsize=(12, 8))

    # Title
    plt.suptitle("Unicode to Preeti Converter - Test Results Summary", fontsize=16, y=0.98)

    # Metrics Results
    plt.subplot(2, 2, 1)
    plt.bar(['CER', 'BLEU'], [cer, bleu], color=['red', 'green'])
    plt.title('Metrics Evaluation')
    plt.ylim(0, 1)

    # Add text display of metrics
    plt.text(0, cer + 0.05, f'{cer:.4f}', ha='center')
    plt.text(1, bleu + 0.05, f'{bleu:.4f}', ha='center')

    # Add note about metric interpretation
    plt.figtext(0.25, 0.4, "CER: Lower is better\nBLEU: Higher is better",
                bbox=dict(facecolor='yellow', alpha=0.2))

    # Add placeholder for qualitative results
    plt.subplot(2, 2, 2)
    plt.axis('off')
    plt.title('Qualitative Assessment')
    plt.text(0.5, 0.5, "See 'qualitative_assessment.png'\nfor detailed results",
             ha='center', va='center', fontsize=10,
             bbox=dict(facecolor='lightblue', alpha=0.4))

    # Add placeholder for performance results
    plt.subplot(2, 2, 3)
    plt.axis('off')
    plt.title('Performance Benchmark')
    plt.text(0.5, 0.5, "See 'performance_benchmark.png'\nfor detailed results",
             ha='center', va='center', fontsize=10,
             bbox=dict(facecolor='lightgreen', alpha=0.4))

    # Add recommendations based on test results
    plt.subplot(2, 2, 4)
    plt.axis('off')
    plt.title('Recommendations')

    # Generate recommendations based on metrics
    recommendations = []
    if cer < 0.2:
        recommendations.append("✓ CER is good (<0.2)")
    else:
        recommendations.append("✗ CER needs improvement (>0.2)")

    if bleu > 0.7:
        recommendations.append("✓ BLEU score is good (>0.7)")
    else:
        recommendations.append("✗ BLEU score needs improvement (<0.7)")

    recommendations.append("➤ See detailed reports for more insights")

    # Display recommendations
    y_pos = 0.8
    for rec in recommendations:
        plt.text(0.1, y_pos, rec, fontsize=10)
        y_pos -= 0.2

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig('test_summary.png')
    plt.close()

    print("\nTest summary visualization saved as 'test_summary.png'")

# Run all tests
if __name__ == "__main__":
    run_comprehensive_tests()


========== UNICODE TO PREETI CONVERTER TESTING ==========

[TEST 1] CER/BLEU Metrics Evaluation


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!



--- CER/BLEU Evaluation ---
Unicode         Expected        Predicted       CER        BLEU      
-----------------------------------------------------------------


/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.11/dist-packages/nltk/translate/bleu_score.py:577: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

क               s               s               0.0000      0.0000
का              sf              sf              0.0000      0.0000
कि              ls              ls              0.0000      0.0000
नमस्ते          gd:t]           gd:t]           0.0000      1.0000
-----------------------------------------------------------------
Average CER: 0.0000 (lower is better)
Average BLEU: 0.2500 (higher is better)

[TEST 2] Qualitative Assessment

--- Qualitative Assessment ---

Category: Simple
Unicode              Preeti              
----------------------------------------
क                    s                   
ख                    v                   
ग                    u                   
घ                    3                   

Category: Medium
Unicode              Preeti              
----------------------------------------
नमस्ते               gd:t]               
कस्तो छ              s:tf] 5             
धन्यवाद              wGojfb              

Category: Complex
Unicode 

<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2325 (\N{DEVANAGARI LETTER KA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Matplotlib currently does not support Devanagari natively.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2326 (\N{DEVANAGARI LETTER KHA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2327 (\N{DEVANAGARI LETTER GA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2328 (\N{DEVANAGARI LETTER GHA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2344 (\N{DEVANAGARI LETTER NA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10-cff367787a41>:53: UserWarning: Glyph 2350 (\N{DEVANAGARI LETTER MA}) missing from font(s) DejaVu Sans.
  plt.tight_layout()
<ipython-input-10


Qualitative assessment visualization saved as 'qualitative_assessment.png'

[TEST 3] Performance Benchmark

--- Performance Benchmark ---
Type                 Length     Time (ms)      
---------------------------------------------
Single Character     1          184.88
Word                 6          386.03
Short Sentence       16         820.61
Medium Sentence      29         1503.17
Long Sentence        115        4853.59

Performance benchmark visualization saved as 'performance_benchmark.png'

Test summary visualization saved as 'test_summary.png'

========== TESTING COMPLETED ==========
